In [1]:
from openai import OpenAI

In [8]:
from openai import OpenAI
client = OpenAI(
    base_url="http://g04:8094/v1",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", role='assistant', function_call=None, tool_calls=[])


In [9]:
import json


data = []

with open("/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+gpt4o-dfewshot-tiebrk-atype/batch_1.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
completion = client.chat.completions.create(
  model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

In [28]:
completion = client.chat.completions.create(model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4", 
                                            messages=data[0]["body"]["messages"], 
                                            temperature=data[0]["body"]["temperature"])

TypeError: Completions.create() got an unexpected keyword argument 'custom_id'

In [25]:
print(completion)

ChatCompletion(id='chat-380a567ac8164511b8bc202b8366b71c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n    "questions":\n        [\n            {"question": "Is there a letter from Sean Connery to Steve Jobs?", "answer": "Yes", "source": "1", "answer_type": "Boolean"},\n            {"question": "What is the content of the letter from Sean Connery to Steve Jobs?", "answer": "The letter claims that Sean Connery refused to appear in an Apple commercial, stating that he has no interest in \'changing the world\' as Steve Jobs suggested, and that he is James Bond, not a computer salesman.", "source": "1", "answer_type": "Extractive"},\n            {"question": "Is the letter from Sean Connery to Steve Jobs real?", "answer": "No", "source": "2", "answer_type": "Boolean"},\n            {"question": "Who created the fake letter from Sean Connery to Steve Jobs?", "answer": "Scoopertino, a satirical site that spoofs Apple news",

In [19]:
print(completion.choices[0].message.content)

```json
{
    "questions":
        [
            {"question": "Is there a letter from Sean Connery to Steve Jobs?", "answer": "Yes", "source": "1", "answer_type": "Boolean"},
            {"question": "What is the content of the letter from Sean Connery to Steve Jobs?", "answer": "The letter claims that Sean Connery refused to appear in an Apple commercial, stating that he has no interest in 'changing the world' as Steve Jobs suggested, and that he is James Bond, not a computer salesman.", "source": "1", "answer_type": "Extractive"},
            {"question": "Is the letter from Sean Connery to Steve Jobs real?", "answer": "No", "source": "2", "answer_type": "Boolean"},
            {"question": "Who created the fake letter from Sean Connery to Steve Jobs?", "answer": "Scoopertino, a satirical site that spoofs Apple news", "source": "2", "answer_type": "Extractive"},
            {"question": "What was the purpose of the fake letter from Sean Connery to Steve Jobs?", "answer": "To accompan

In [36]:
import os
import pickle
from openai import OpenAI
client = OpenAI(
    base_url="http://g04:8094/v1",
    api_key="token-abc123",
)

#files in directory
DIR = '/mnt/data/factcheck/averitec-data/data_store/batch_jobs/dev_mmr+gpt4o-dfewshot-tiebrk-atype'
OUT_DIR = "/mnt/data/factcheck/averitec-data/data_store/batch_jobs_llama_res/dev_mmr+gpt4o-dfewshot-tiebrk-atype"
files = os.listdir(DIR)
sorted_files = sorted(files)
#leave only files with batch prefix
sorted_files = [f for f in sorted_files if f.startswith("batch")]

results = {}
for f in sorted_files:
    with open(os.path.join(DIR, f), "r") as file:
        for line in file:
            data = json.loads(line)
            completion = client.chat.completions.create(model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4", 
                                                        messages=data["body"]["messages"], 
                                                        temperature=data["body"]["temperature"])
            results[data["custom_id"]] = completion
            break
        

    #save results to pickle
    with open(os.path.join(OUT_DIR, f[:-6] + "_" + "results.pickle"), "wb") as out_f:
        pickle.dump(results, out_f)

    #save results to jsonl
    with open(os.path.join(OUT_DIR, f[:-6] + "_" + "results.jsonl"), "w") as out_f:
        for k, v in results.items():
            out_f.write(json.dumps({"custom_id": k, "response": v.choices[0].message.content}) + "\n")
    break


